## 2) Data Collection and Pre-Processing
- I am using the .txt files from the following books: 
- I am combining them in a list and pre-processing them
-

In [ ]:
import string
import nltk
import certifi
import re
import gensim
from gensim.models import Word2Vec
import contractions
import unicodedata
import os
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from flask import Flask, request

In [ ]:
os.environ['SSL_CERT_FILE'] = certifi.where()
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
books = ['adventures.txt', 'crime_and_punishment.txt', 'don_quijote.txt', 'frankenstein.txt', 'great_expectations.txt', 'history_of_tom_jones.txt', 'jane_eyre.txt', 'leviathan.txt', 'makers.txt', 'middlemarch.txt', 'moby_dick.txt', 'my_life.txt', 'pride_and_prejudice.txt', 'shakespeare_complete_works.txt', 'the_count_of_monte_cristo.txt', 'twenty_years_after.txt', 'ulysses.txt', 'war_and_peace.txt']

for txt_file in books:
    with open(txt_file, 'r', encoding = 'utf-8') as file:
        content = file.read()

In [ ]:
def convert_utf(text):
    text = text.replace('\u2018', "'").replace('\u2019', "'").replace('\u201C', "`").replace('\u201D', "`").replace('\u2013', '-').replace('\u2014', '-')
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore')
    return text.decode('ascii')

In [ ]:
all_sentences = []

for txt_file in books:
    with open(txt_file, 'r', encoding='utf-8') as file:
        content = file.read()

        # Converting UTF characters
        content = convert_utf(content)
        # Lowercasing
        content = content.lower()
        # Expanding contractions
        content = ' '.join([contractions.fix(word) for word in content.split()])

        # Sentence tokenization
        sentences = sent_tokenize(content)

        for sentence in sentences:
            # Word tokenization
            words = word_tokenize(sentence)

            # Remove punctuation
            words = [word for word in words if word.isalpha()]

            all_sentences.append(words)

In [ ]:
# Checking the first few processed sentences:

for i in range(5):
    print(f"Sentence {i+1}: {all_sentences[i]}")

In [ ]:
# Checking if any character is not a lowercase letter:

for sentence in all_sentences:
    for word in sentence:
        if not word.islower():
            print(f"Non-lowercase word found: {word}")

# Print a message if no issues are found
print("No non-lowercase words found.")

## 3) Training[20marks]:
● Use a Word2Vec embeddings technique. (10 marks)
● Utilise Gensim library to assist with the training.
● Save the trained model for future use. (10 marks)

#### Step 1: Import Gensim and Prepare the Model
- First, import the necessary Gensim modules and create an instance of the Word2Vec model.vector_size: The number of dimensions of the embeddings.
- window: The maximum distance between the current and predicted word within a sentence.
- min_count: Ignores all words with total frequency lower than this.
- sg: Training algorithm: 1 for skip-gram; otherwise CBOW.

In [ ]:
# Initialize Word2Vec model
model = gensim.models.Word2Vec(vector_size=200, window=5, min_count=1, sg=0)

#### Step 2: Build Vocabulary
- Load your preprocessed data (the all_sentences list) and build the model's vocabulary.
- all_sentences is the list of tokenized sentences you prepared.
- update=False means the vocabulary is initialized for the first time.

In [ ]:
model.build_vocab(all_sentences, update=False)

#### Step 3: Train the Model with the sentences.
- total_examples: Count of sentences.
- epochs: Number of iterations over the corpus.

In [ ]:
model.train(all_sentences, total_examples=model.corpus_count, epochs=model.epochs)

#### Step 4: Save the Model
-Save the trained model for future use.

In [ ]:
model.save('model_su')

#### Step 5: Test the Model
- Check how the model performs with an example.
- This prints the words most similar to 'man'.

In [ ]:
print(model.wv.most_similar('man', topn=5))

Below I have the same preprocessing for the model, but it additionally removes stopwords. I want to see if my model is more accurate with or without them.

In [ ]:
# OPTIONAL

all_sentences_no_stopwords = []
counter = 0
for txt_file in books:
    counter += 1
    print(counter)
    with open(txt_file, 'r', encoding='utf-8') as file:
        content = file.read()

        # Converting UTF characters and preprocess
        content = convert_utf(content)
        content = content.lower()

        # Sentence tokenization
        sentences = sent_tokenize(content)

        for sentence in sentences:
            # Word tokenization
            words = word_tokenize(sentence)

            # Removing punctuation and stopwords
            words = [word for word in words if word.isalpha() and word not in stopwords.words('english')]

            all_sentences_no_stopwords.append(words)

In [ ]:
model.build_vocab(all_sentences_no_stopwords, update=False)
model.train(all_sentences, total_examples=model.corpus_count, epochs=model.epochs)
model.save('model_su_no_stop')
